In [3]:
import ir_datasets
import pandas as pd

dataset = ir_datasets.load("cord19")

data = []

for doc in dataset.docs_iter()[:20000]:  
 
    data.append({
        "doc_id": doc.doc_id,
        "title": doc.title,
        "doi": doc.doi,
        "date": doc.date,
        "abstract": doc.abstract
    })

df = pd.DataFrame(data)

df.to_csv("covid_dataset.csv", index=False)

print("Dataset saved to covid_dataset.csv")

Dataset saved to covid_dataset.csv
